In [1]:
import pandas as pd
import string
import re
import datetime
import collections
import math
import heapq
import pickle 

## marks lists


In [2]:
punctuation_marks = [')','(','>','<',"؛","،",'{','}',"؟",':',"-", '»', '"', '«', '[', ']','"','+','=','?']
marks = ['/','//', '\\','|','!', '%', '&','*','$', '#','؟', '*','.','_' ]
alphabet_string_lower = string.ascii_lowercase
alphabet_string_upper = string.ascii_uppercase
english_char =  list(alphabet_string_lower) + list(alphabet_string_upper)


sep_list = [" ", '\xad', '\u200e','\u200f', '\u200d', '\u200d', '\u200d'] + marks



## regex  patterns, prefixe list and postfix list

In [3]:


raw_postfix = [
        ('[\u200c](تر(ین?)?|گری?|های?)' , ''),
        (r'(تر(ین?)?|گری?)(?=$)' , ''), #تر ، ترین، گر، گری در آخر کلمه
        (r'(?<=[^ او])(م|ت|ش|مان|تان|شان)$' , ''), # حذف شناسه های مالکیت و فعل در آخر کلمه
        (r'(ان|ات|گی|گری|های)$' , ''), #ان، ات، ها، های آخر کلمه  
]

raw_arabic_notation = [
    # remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
    ('[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652]', ''),
    ]

raw_long_letters = [
    (r' +', ' '),  # remove extra spaces
    (r'\n\n+', '\n'),  # remove extra newlines
    (r'[ـ\r]', ''),  # remove keshide, carriage returns
    ]

raw_half_space = [
    ('[\u200c]', ''),
]
 

## verbs, mokassar,morakab list

In [4]:
present_roots =['توان','باش','رو','بر','یاور', 'یانداز', 'یای','یاندیش','بخش','باز','خر','بین','شنو','دار','دان','رسان','شناس','گو','گذار','یاب','لرز','ساز','شو','نویس','خوان','کاه','گیر','خواه','کن' ]

past_roots = ['توانست','بود','کرد','اورد','انداخت','امد','خرید','باخت','برد','رفت','اندیشید','بخشید','دید','شنید','داشت','دانست','رساند','شناخت','گفت','گذشت','یافت','لرزید','ساخت','شد','نوشت','خواند','کاست','گرفت','خواست']
all_verbs_roots = present_roots + past_roots
empty_list = ['','','','','','','']
verb_prefix = ['نمی‌', 'می‌','ن','ب',"" ]
present_verb_postfix = ['م','ی','د','ید','ند','یم']
past_verb_postfix = ['ایم','اید','ای','ام','اند']
past_verb_postfix2 = ['م','ی','ید','ند','یم']
present_verbs = []
past_verbs = []
all_verbs = {}
for pref in verb_prefix:
    for present_root, past_root in zip(present_roots, past_roots):
        for post in past_verb_postfix2:
            all_verbs[pref + past_root+post] = past_root
        for post in past_verb_postfix:
            all_verbs[past_root+'ه‌'+post] = past_root
        for post in present_verb_postfix:
            all_verbs[pref+present_root+post] = present_root
            
words_list = ['ادب', 'آداب', 'طرف', 'اطراف', 'حقیقت', 'حقایق', 'موج', 'امواج', 'ادیب', 'ادبا', 'عمق', 'اعماق', 'خزینه', 'خزائن', 'مرکز', 'مراکز', 'اثر', 'آثار', 'عالم', 'علما', 'خبر', 'اخبار', 'موقع', 'مواقع', 'اسیر', 'اسرا', 'علم', 'علوم', 'دوره', 'ادوار', 'مصرف', 'مصارف', 'اسم', 'اسامی', 'علامت', 'علائم', 'دین', 'ادیان', 'معرفت', 'معارف', 'اسطوره', 'اساطیر', 'علت', 'علل', 'دفتر', 'دفاتر', 'مبحث', 'مباحث', 'امیر', 'امرا', 'عقیده', 'عقائد', 'ذخیره', 'ذخایر', 'ماده', 'مواد', 'امر', 'اوامر', 'عمل', 'اعمال', 'رفیق', 'رفقا', 'مذهب', 'مذاهب', 'امام', 'ائمه', 'عید', 'اعیاد', 'رای', 'آرا', 'مصیبت', 'مصائب', 'اصل', 'اصول', 'عنصر', 'عناصر', 'رسم', 'رسوم', 'معبد', 'معابد', 'افق', 'آفاق', 'عاطفه', 'عواطف', 'رابطه', 'روابط', 'مسجد', 'مساجد', 'بیت', 'ابیات', 'عضو', 'اعضا', 'رمز', 'رموز', 'معبر', 'معابر', 'تاجر', 'تجار', 'عبارت', 'عبارات', 'رجل', 'رجال', 'مظهر', 'مظاهر', 'تصویر', 'تصاویر', 'عجیب', 'عجایب', 'رقم', 'ارقام', 'منظره', 'مناظر', 'جد', 'اجداد', 'فقیه', 'فقها', 'زاویه', 'زوایا', 'مرض', 'امراض', 'جانب', 'جوانب', 'فن', 'فنون', 'زعیم', 'زعما', 'مورد', 'موارد', 'جزیره', 'جزایر', 'فکر', 'افکار', 'سانحه', 'سوانح', 'مرحله', 'مراحل', 'جبل', 'جبال', 'فریضه', 'فرایض', 'سلطان', 'سلاطین', 'مفهوم', 'مفاهیم', 'جریمه', 'جرایم', 'فعل', 'افعال', 'شعر', 'اشعار', 'منبع', 'منابع', 'حادثه', 'حوادث', 'فقیر', 'فقرا', 'شاعر', 'شعرا', 'مکان', 'اماکن', 'حشم', 'احشام', 'قاعده', 'قواعد']
mokassar_dict ={}
for i in range(0,len(words_list),2):
    mokassar_dict[words_list[i+1]] = words_list[i]
                                       

## tokenizer

In [5]:
def split(txt, seps):
    default_sep = seps[0]    
    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]

#get doc and return (word,doc id)
def tokenizer(text):
    for l in english_char:
        text = text.replace(l,"")
    word_list = split(text.strip().replace("\n"," "), sep_list)
    word_list = [x for x in word_list if not x.startswith("ir") and not x.startswith("NID")]
    return word_list

## normalizer

In [6]:
def remove_punctuation_marks(word_list):
#     word_list = list(word_list)
    for i in range(len(word_list)):
        for mark in punctuation_marks:
            word_list[i] = word_list[i].replace(mark,"")

    return word_list


def edit_long_letters(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_long_letters]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list




def remove_arabic_notation(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_arabic_notation]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list


def create_translation_table(src_list, dst_list):
     return dict((ord(a), b) for a, b in zip(src_list, dst_list)) #map src unicode to dst char

    
def char_digit_Unification(word_list): 
    word_list = [x for x in word_list if not x.isnumeric() or (x.isnumeric() and float(x)<3000)]
    translation_src, translation_dst = ' ىكي“”', ' یکی""'
    translation_src += 'ئ0123456789أإآ%'
    translation_dst += 'ی۰۱۲۳۴۵۶۷۸۹ااا٪'
    translations_table = create_translation_table(translation_src, translation_dst)
    for i in range(len(word_list)):
        word_list[i] = word_list[i].translate(translations_table)
    return word_list

def remove_mokassar(word_list):
    for i in range(len(word_list)):
        word_list[i] = mokassar_dict.get(word_list[i],word_list[i])
        
    return word_list
    


def verb_Steaming(word_list):
    for i in range(len(word_list)):
        word_list[i] = all_verbs.get(word_list[i],word_list[i])
        
    return word_list
    

def remove_postfix(word_list):

    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_postfix]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            if len(word_list[i])>4 and  not (word_list[i] in all_verbs_roots):
                word_list[i] = pattern.sub(rep, word_list[i])
    return word_list


    
    
def remove_prefix(word_list):
    starts =['بی‌' , 'نا', 'با']
    for i in range(len(word_list)):
        for start in starts:
            if word_list[i].startswith(start) and len(word_list[i])>4:
                word_list[i] = word_list[i][len(start):]
                break

    return word_list

def morakab_Unification(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_half_space]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list

#all to gether 
def normalizer(word_list):
    #remove punc
    for i in range(len(word_list)):
        for mark in punctuation_marks:
            word_list[i] = word_list[i].replace(mark,"")
            
    #edit long letters
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_long_letters]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
            
    #remove mokassar
    for i in range(len(word_list)):
        word_list[i] = mokassar_dict.get(word_list[i],word_list[i])
    
    #remove_arabic_notation
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_arabic_notation]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    
    #char_digit_Unification
    word_list = [x for x in word_list if not x.isnumeric() or (x.isnumeric() and float(x)<3000)]
    translation_src, translation_dst = ' ىكي“”', ' یکی""'
    translation_src += 'ئ0123456789أإآ%'
    translation_dst += 'ی۰۱۲۳۴۵۶۷۸۹ااا٪'
    translations_table = create_translation_table(translation_src, translation_dst)
    for i in range(len(word_list)):
        word_list[i] = word_list[i].translate(translations_table)
    
    #verb_Steaming
    for i in range(len(word_list)):
        word_list[i] = all_verbs.get(word_list[i],word_list[i])
        
    #remove_prefix
    starts =['بی‌' , 'نا', 'با']
    for i in range(len(word_list)):
        for start in starts:
            if word_list[i].startswith(start) and len(word_list[i])>4:
                word_list[i] = word_list[i][len(start):]
                break
    
    #remove_postfix
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_postfix]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            if len(word_list[i])>4 and  not (word_list[i] in all_verbs_roots):
                word_list[i] = pattern.sub(rep, word_list[i])
    #morakab_Unification
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_half_space]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list

## read data

In [12]:
%%time
cols = ['id']
df = pd.read_excel('IR_Spring2021_ph12_7k.xlsx', engine = 'openpyxl')
contents = df['content'].to_list()
doc_ids = df['id']
doc_url = df['url']
docs_num = len(doc_ids)


Wall time: 855 ms


## create inverted index

In [13]:
def create_champion_list(inverted_index, r):
    for term in inverted_index.keys():
        doc_freq_dict = inverted_index[term]["normal"]
        heap = [(-value, key) for key,value in doc_freq_dict.items()]
        largest = heapq.nsmallest(r, heap)
        champion_list = { y:-x for x,y in largest}
        champion_list = sorted(champion_list.items(), key=lambda kv: kv[0])
        inverted_index[term]["champion"] = {key:value for key, value in champion_list}
    

def create_dictionary(contents, doc_ids, r):
    for content, id in zip(contents,doc_ids):
        doc_tokens = tokenizer(content)
        doc_normalized_tokens = normalizer(doc_tokens)
        term_freq_dict = dict(collections.Counter(doc_normalized_tokens))
        # we replace contents by bag of word style doc : { term -> freq}
        contents[id-1] = term_freq_dict
        for term,freq in term_freq_dict.items():
            term_postings = inverted_index.get(term,{"normal":{}, "champion":{}})
            term_postings["normal"][id] = freq
#             term_postings["champion"][id] = freq
            inverted_index[term] = term_postings
    # term -> {normal : list of docs, champion: list of champion docs(length of the lis is r)}
    create_champion_list(inverted_index, r)
    

def delete_soptwords(inverted_index):
    stop_words = []
    for key in list(inverted_index):
        if len(inverted_index[key]["normal"])> 0.25* docs_num or len(key) <2: 
            stop_words.append(key)
            del inverted_index[key]
    return stop_words
    



In [14]:
%%time
inverted_index ={}
create_dictionary(contents, doc_ids, 200)
stop_words = delete_soptwords(inverted_index)
print("stop words: ", stop_words )



stop words:  ['به', 'گزار', 'ایسنا', 'از', 'ایر', 'با', 'این', 'هم', 'داد', '', 'که', 'شده', 'حضور', 'را', 'در', 'های', 'کن', 'است', 'سال', 'خود', 'کرد', 'عنو', 'بود', 'باش', 'اعلام', 'می', 'بر', 'و', 'شد', 'نیز', 'برای', 'خواه', 'کشور', 'روز', 'همچنین', 'کرده', 'اینکه', '۳', 'انت', 'پیام', 'بیان', 'شو', 'گفت', 'توان', 'وی', 'افزود', 'ساز', 'توجه', 'اظهار', 'تا', '۲', 'داشت', 'باید', 'مورد', 'انجام', 'دار', 'گیر', 'دو', 'هر', 'ان', 'رو', 'یا', 'صورت', 'یک', 'ی', 'اما', '٢', 'گذشته', 'هزار', '۴', 'قرار', 'بیش', 'دیگر', 'ادامه', 'امروز', '۰', '۹', '۷', '۵', '۶', '۱', '۸', 'ا', '٣', 'ب', 'س', '–', '٨', 'ج', '٧', 'پ', "'", 'ز', 'ر', 'ه', 'ع', 'ء', 'م', '\u202c', 'ش', '٦', 'ن', '٩', '@', 'ۚ', '٤', 'د', 'ص', 'ک', '\u202b', 'ث', 'چ', 'ط', 'ح', 'ل', 'ف', 'ت', '…', '٥', '\u2067', '١', 'ذ', 'ق', '—', '٠', ',', 'غ', '\ufeff', '٪', '"', '\u2069', '•']
Wall time: 23.2 s


In [49]:
# print(len(inverted_index['ملی']["champion"]))
# print(((inverted_index['ملی']["champion"])))
# print("/*****************")
# print((inverted_index['ملی']["normal"]))
print(len(inverted_index['سلام']['champion']))

200


## save inverted_index

In [10]:
%%time
file_inverted_index = open('ph2_inverted_index.obj', 'wb') 
pickle.dump(inverted_index, file_inverted_index)
file_inverted_index.close()
inverted_index = {}


Wall time: 136 ms


In [51]:
file_contents = open('ph2_contents.obj', 'wb') 
pickle.dump(contents, file_contents)
file_contents.close()
contents =[]

## load inverted_index

In [52]:
%%time
readed = open('ph2_inverted_index.obj', 'rb') 
inverted_index = pickle.load(readed)
readed = open('ph2_contents.obj', 'rb') 
contents = pickle.load(readed)

Wall time: 426 ms


## vectorize query

In [15]:
def vectorize_query(query_str):
    # free_text query => {term: tf}
    query_tokens = tokenizer(query_str)
    query_normalized_tokens = normalizer(query_tokens)
    print(query_normalized_tokens)
    term_freq_dict = dict(collections.Counter(query_normalized_tokens))
    query_terms = {}
    for  term,freq in term_freq_dict.items():
        if term in inverted_index.keys() and term not in stop_words:
            query_terms[term] = 1+math.log(freq,10)
    return query_terms
    

## query index elimination

In [16]:
def score_docs(query_terms, postings_type = "normal"):
    # method parameters define we should iterate on which postings list
    docs_score = [0] * (docs_num+1)
    # we use TAAT method and score all docs which have the term on each iterate 
    for term,Wt_q in query_terms.items():
        idf = math.log(docs_num/len(inverted_index[term]['normal']))
        if postings_type == "normal":
            for doc_id, freq in inverted_index[term]["normal"].items():
                docs_score[doc_id] += Wt_q * (1+math.log(freq,10)) * idf
        elif postings_type == "champion":
            for doc_id, freq in inverted_index[term]["champion"].items():
                docs_score[doc_id] += Wt_q * (1+math.log(freq,10)) * idf
    return docs_score
            

In [17]:
def calculate_length(doc_id):
    sum =0
    for term, freq in contents[doc_id-1].items():
        if term not in stop_words:
            # sum += (tf * idf)**2
            sum += ((1+(math.log(freq,10))) * math.log(docs_num/len(inverted_index[term]['normal']),10) )**2
    return math.sqrt(sum)

In [18]:
def similarity(docs_score):
    docs_similarity = {}
    for doc_id in range(1,docs_num+1):
        if docs_score[doc_id] != 0:
            docs_similarity[doc_id] = docs_score[doc_id]/calculate_length(doc_id)
    return docs_similarity
            

In [19]:
def query_index_elimination(query_str, postings_type="normal" ):
    query_terms = vectorize_query(query_str)
    docs_score = score_docs(query_terms, postings_type)
    docs_similarity = similarity(docs_score)
    return docs_similarity 
    

## top-k extraction

In [20]:
def extract_top_k(docs_similarity, k, method = "heap"):
    #heap method
    if method == "heap": 
        heap = [(-value, key) for key,value in docs_similarity.items()]
        largest = heapq.nsmallest(k, heap)
        top_k = [(y, -x) for x,y in largest]
        return top_k
    
    # sort method
    else:
        docs_similarity_list = list(docs_similarity.items())
        for mx in range(len(docs_similarity_list)-1, -1, -1):
            swapped = False
            for i in range(mx):
                if docs_similarity_list[i][1] < docs_similarity_list[i+1][1]:
                    docs_similarity_list[i], docs_similarity_list[i+1] = docs_similarity_list[i+1], docs_similarity_list[i]
                    swapped = True
            if not swapped:
                break
        top_k = docs_similarity_list[:k]
        return top_k


## query responding

In [28]:
def query_responding(postings_type, top_k_method):
    query = input("enter query: ")
    a = datetime.datetime.now()
    docs_similarity = query_index_elimination(query, postings_type)
    top_docs = extract_top_k(docs_similarity, 50, top_k_method )
    b = datetime.datetime.now()


    print("{:<5} result in {} ms\nid \t(score)\t\t  -> link\n".format(len(top_docs),1000*(b-a).total_seconds()))
    for doc_id, score in top_docs:
        print( "{:<5}({}) -> {} ".format(doc_id,score, doc_url[doc_id-1]))

In [31]:
query_responding("champion","heap")

enter query: تحرک ذهنی زنان خانه‌دار
['تحرک', 'ذهنی', 'زنان', 'خانهدار']
50    result in 147.593 ms
id 	(score)		  -> link

6718 (1.0697567199327356) -> https://www.isna.ir/news/98042915171/زنان-شاغل-دیرتر-به-آلزایمر-مبتلا-می-شوند 
6722 (1.0697567199327356) -> https://www.isna.ir/news/98042915171/زنان-شاغل-دیرتر-به-آلزایمر-مبتلا-می-شوند 
1606 (0.5457466377911007) -> https://www.isna.ir/news/98111309170/سفر-تیم-ملی-فوتسال-بانوان-چین-به-ایران-لغو-شد 
2733 (0.46947215051185165) -> https://www.isna.ir/news/98042614092/تحرک-نیروی-زمینی-نسبت-به-سال-گذشته-30-درصد-افزایش-داشته-است 
1921 (0.4686017778103753) -> https://www.isna.ir/news/99050100921/جلسه-شورای-مرکزی-مجمع-زنان-اصلاح-طلب-برگزار-شد 
3038 (0.46605868065945855) -> https://www.isna.ir/news/98091309931/بسیج-برنامه-انتخاباتی-برای-مجلس-ندارد 
6921 (0.4593294302330212) -> https://www.isna.ir/news/98081308061/حواس-افسردگی-را-پرت-کنید 
6924 (0.4593294302330212) -> https://www.isna.ir/news/98081308061/حواس-افسردگی-را-پرت-کنید 
6928 (0.4593294

In [24]:
# query_responding("champion","other")

In [25]:
query_responding("normal", "heap")

enter query: آلودگی هوا محیط زیست
['الود', 'هوا', 'محیط', 'زیست']
50    result in 289.928 ms
id 	(score)		  -> link

4166 (-) -> https://www.isna.ir/news/99110906805/آلایندگی-شدید-یکی-از-پالایشگاه-های-عسلویه-به-دلیل-قطعی-برق 
3583 (-) -> https://www.isna.ir/news/99042820529/محکومیت-سه-پالایشگاه-گازی-به-علت-آلودگی-خلیج-فارس 
4704 (-) -> https://www.isna.ir/news/98042513336/شکایت-محیط-زیست-از-پالایشگاهی-در-جنوب-بوشهر 
833  (-) -> https://www.isna.ir/news/99102116054/در-چه-صورتی-شهرآورد-پایتخت-لغو-می-شود 
4240 (-) -> https://www.isna.ir/news/99120503925/تلف-شدن-ماهی-ها-در-سواحل-عسلویه 
4380 (-) -> https://www.isna.ir/news/98020402451/قیر-حکم-طلا-پیدا-کرده 
4099 (-) -> https://www.isna.ir/news/99101410310/سوخت-توزیع-شده-در-تهران-دارای-استاندارد-های-لازم-است 
1321 (-) -> https://www.isna.ir/news/98060603293/شکایت-از-دوچرخه-سواری-در-لندن-بالا-گرفته 
4126 (-) -> https://www.isna.ir/news/99102418523/بلایی-که-پتروشیمی-ایلام-بر-سر-هوا-آورده-است 
6281 (-) -> https://www.isna.ir/news/99092620747/۹

In [26]:
# query_responding("normal", "other")